In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [2]:
df = pd.read_csv("C:/Users/Ankan SInha/Desktop/Finl 2/Data/I/ceec7ed40f2ddeebd501ba2b785ff305.csv")
df

,Tweets
0,"#USER# #USER# #USER# #USER# #USER# I'd be shocked if he ""pushed"" a theory that there was little ..."
1,#USER# #USER# Maybe that was off set by his incompetency and danger to global stability? idk
2,"#USER# #USER# Select doctors d not represent the medical community. Also, does that mean that re..."
3,#USER# #USER# #USER# #USER# That there are complicated cases does not invalidate the point. To s...
4,#USER# #USER# #USER# Metanarrative in the sense used in postmodernism is just that - a lens thro...
5,#USER# Face it Fox News! For some obscene reason the vice president does not take directions fro...
6,#USER# #USER# #USER# There really is not much more I can say if you think reports from the CDC a...
7,#USER# #USER# I can honestly say I've never been accused of being a sheep or anything like it by...
8,"#USER# Also, live in a lawless country like the USA. Or, at least lawless for the wealthy."
9,"#USER# #USER# Yes, there is no rational argument against abortion. So are you saying that early ..."


In [3]:
for col in df.columns:
    print(col, df[col].isnull().sum())

Tweets 0


In [4]:
rws = df.loc[:, ['Tweets']]

In [5]:
rws['Tweets_token'] = rws['Tweets'].apply(lambda x: [contractions.fix(word) for word in x.split()])
rws.head()

,Tweets,Tweets_token
0,"#USER# #USER# #USER# #USER# #USER# I'd be shocked if he ""pushed"" a theory that there was little ...","[#USER#, #USER#, #USER#, #USER#, #USER#, I would, be, shocked, if, he, ""pushed"", a, theory, that..."
1,#USER# #USER# Maybe that was off set by his incompetency and danger to global stability? idk,"[#USER#, #USER#, Maybe, that, was, off, set, by, his, incompetency, and, danger, to, global, sta..."
2,"#USER# #USER# Select doctors d not represent the medical community. Also, does that mean that re...","[#USER#, #USER#, Select, doctors, d, not, represent, the, medical, community., Also,, does, that..."
3,#USER# #USER# #USER# #USER# That there are complicated cases does not invalidate the point. To s...,"[#USER#, #USER#, #USER#, #USER#, That, there, are, complicated, cases, does, not, invalidate, th..."
4,#USER# #USER# #USER# Metanarrative in the sense used in postmodernism is just that - a lens thro...,"[#USER#, #USER#, #USER#, Metanarrative, in, the, sense, used, in, postmodernism, is, just, that,..."


In [6]:
rws['rating_description_str'] = [' '.join(map(str, l)) for l in rws['Tweets_token']]
rws['tokenized'] = rws['rating_description_str'].apply(word_tokenize)
rws['lower'] = rws['tokenized'].apply(lambda x: [word.lower() for word in x])
punc = string.punctuation
rws['no_punc'] = rws['lower'].apply(lambda x: [word for word in x if word not in punc])
stop_words = set(stopwords.words('english'))
new_stopwords = ["#", "url", "user",'a','an','the','of','or','is','it','for','have','has']
stopwrd = nltk.corpus.stopwords.words('english')
stopwrd.extend(new_stopwords)

rws['stopwords_removed'] = rws['no_punc'].apply(lambda x: [word for word in x if word not in new_stopwords])
rws['pos_tags'] = rws['stopwords_removed'].apply(nltk.tag.pos_tag)
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
rws['wordnet_pos'] = rws['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
wnl = WordNetLemmatizer()
rws['lemmatized'] = rws['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word) for (word, tag) in x])
rws.head()

,Tweets,Tweets_token,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,"#USER# #USER# #USER# #USER# #USER# I'd be shocked if he ""pushed"" a theory that there was little ...","[#USER#, #USER#, #USER#, #USER#, #USER#, I would, be, shocked, if, he, ""pushed"", a, theory, that...","#USER# #USER# #USER# #USER# #USER# I would be shocked if he ""pushed"" a theory that there was lit...","[#, USER, #, #, USER, #, #, USER, #, #, USER, #, #, USER, #, I, would, be, shocked, if, he, ``, ...","[#, user, #, #, user, #, #, user, #, #, user, #, #, user, #, i, would, be, shocked, if, he, ``, ...","[user, user, user, user, user, i, would, be, shocked, if, he, ``, pushed, '', a, theory, that, t...","[i, would, be, shocked, if, he, ``, pushed, '', theory, that, there, was, little, to, no, eviden...","[(i, NN), (would, MD), (be, VB), (shocked, VBN), (if, IN), (he, PRP), (``, ``), (pushed, JJ), ('...","[(i, n), (would, n), (be, v), (shocked, v), (if, n), (he, n), (``, n), (pushed, a), ('', n), (th...","[i, would, be, shocked, if, he, ``, pushed, '', theory, that, there, wa, little, to, no, evidenc..."
1,#USER# #USER# Maybe that was off set by his incompetency and danger to global stability? idk,"[#USER#, #USER#, Maybe, that, was, off, set, by, his, incompetency, and, danger, to, global, sta...",#USER# #USER# Maybe that was off set by his incompetency and danger to global stability? i do no...,"[#, USER, #, #, USER, #, Maybe, that, was, off, set, by, his, incompetency, and, danger, to, glo...","[#, user, #, #, user, #, maybe, that, was, off, set, by, his, incompetency, and, danger, to, glo...","[user, user, maybe, that, was, off, set, by, his, incompetency, and, danger, to, global, stabili...","[maybe, that, was, off, set, by, his, incompetency, and, danger, to, global, stability, i, do, n...","[(maybe, RB), (that, DT), (was, VBD), (off, RB), (set, VBN), (by, IN), (his, PRP$), (incompetenc...","[(maybe, r), (that, n), (was, v), (off, r), (set, v), (by, n), (his, n), (incompetency, n), (and...","[maybe, that, wa, off, set, by, his, incompetency, and, danger, to, global, stability, i, do, no..."
2,"#USER# #USER# Select doctors d not represent the medical community. Also, does that mean that re...","[#USER#, #USER#, Select, doctors, d, not, represent, the, medical, community., Also,, does, that...","#USER# #USER# Select doctors d not represent the medical community. Also, does that mean that re...","[#, USER, #, #, USER, #, Select, doctors, d, not, represent, the, medical, community, ., Also, ,...","[#, user, #, #, user, #, select, doctors, d, not, represent, the, medical, community, ., also, ,...","[user, user, select, doctors, d, not, represent, the, medical, community, also, does, that, mean...","[select, doctors, d, not, represent, medical, community, also, does, that, mean, that, restricti...","[(select, JJ), (doctors, NNS), (d, VBP), (not, RB), (represent, JJ), (medical, JJ), (community, ...","[(select, a), (doctors, n), (d, v), (not, r), (represent, a), (medical, a), (community, n), (als...","[select, doctor, d, not, represent, medical, community, also, doe, that, mean, that, restriction..."
3,#USER# #USER# #USER# #USER# That there are complicated cases does not invalidate the point. To s...,"[#USER#, #USER#, #USER#, #USER#, That, there, are, complicated, cases, does, not, invalidate, th...",#USER# #USER# #USER# #USER# That there are complicated cases does not invalidate the point. To s...,"[#, USER, #, #, USER, #, #, USER, #, #, USER, #, That, there, are, complicated, cases, does, not...","[#, user, #, #, user, #, #, user, #, #, user, #, that, there, are, complicated, cases, does, not...","[user, user, user, user, that, there, are, complicated, cases, does, not, invalidate, the, point...","[that, there, are, complicated, cases, does, not, invalidate, point, to, say, someone, looks, ca...","[(that, IN), (there, EX), (are, VBP), (complicated, VBN), (cases, NNS), (does, VBZ), (not, RB), ...","[(that

In [7]:
rws.to_csv('C:/Users/Ankan SInha/Desktop/Finl 2/Data/Trainng Set/i/User_1.csv')

In [8]:
file = open('C:/Users/Ankan SInha/Desktop/Final/Irony/user_processed/lem_i_2.csv', encoding="utf8")
read_data = file.read()
d_per_word = read_data.split()
print('Total Words:', len(d_per_word))

Total Words: 4999


In [9]:
File = open('C:/Users/Ankan SInha/Desktop/Final/Irony/user_processed/User_2.csv',encoding="utf8") #open file
lines = File.read() #read all lines
print (lines)
sentences = nltk.sent_tokenize(lines) #tokenize sentences
nouns = [] #empty to array to hold all nouns

for sentence in sentences:
     for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
             nouns.append(word)
v=list(nouns)
v.remove('\'')
''.join(v)

,Tweets,Tweets_token,rating_description_str,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,"#USER# #USER# #USER# #USER# #USER# I'd be shocked if he ""pushed"" a theory that there was little to no evidence for. Can we walk and chew gum here? There are two concerns, science and ethics. If the evidence supports X you report that fact — also don't hurt people you don't need to hurt.","['#USER#', '#USER#', '#USER#', '#USER#', '#USER#', 'I would', 'be', 'shocked', 'if', 'he', '""pushed""', 'a', 'theory', 'that', 'there', 'was', 'little', 'to', 'no', 'evidence', 'for.', 'Can', 'we', 'walk', 'and', 'chew', 'gum', 'here?', 'There', 'are', 'two', 'concerns,', 'science', 'and', 'ethics.', 'If', 'the', 'evidence', 'supports', 'X', 'you', 'report', 'that', 'fact', '—', 'also', 'do not', 'hurt', 'people', 'you', 'do not', 'need', 'to', 'hurt.']","#USER# #USER# #USER# #USER# #USER# I would be shocked if he ""pushed"" a theory that there was little to no evidence for. Can we walk an

"TweetsTweets_tokenrating_description_strpos_tagswordnet_posUSERUSERUSERUSERUSERtheoryevidencegumconcernsscienceethicsevidenceXfact—people[USERUSERUSERUSERUSER'be'if'he'theory'was'little'to'no'evidence'we'and'chew'gum'are'two'concerns'science'and'ethics'If'the'evidence'supportsX'you'report'fact'—'also'hurt'you'to]USERUSERUSERUSERUSERtheoryevidencegumconcernsscienceethicsevidenceXfact—people['user'user'user'user'useri'be'if'he'theory'was'little'to'no'evidence'for'we'and'chew'gum'are'two'concerns'science'and'ethics'if'the'evidence'supportsx'you'report'fact'—'also'do'not'hurt'you'do'not'to'hurt]['user'user'user'user'useri'be'if'he'theory'was'little'to'no'evidence'for'we'and'chew'gum'are'two'concerns'science'and'ethics'if'the'evidence'supportsx'you'report'fact'—'also'do'not'hurt'you'do'not'to'hurt][i'be'if'he'theory'was'little'to'no'evidence'we'and'chew'gum'are'two'concerns'science'and'ethics'if'evidence'supportsx'you'report'fact'—'also'do'not'hurt'you'do'not'to'hurt][i'NN'MD'VB'VBN'IN'PRP

In [10]:
File = open('C:/Users/Ankan SInha/Desktop/Final/Irony/user_processed/lem_i_2.csv', encoding="utf8") #open file
lines = File.read() #read all lines
sentences = nltk.sent_tokenize(lines) #tokenize sentences
nouns = [] #empty to array to hold all nouns

for sentence in sentences:
     for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
         if (pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS'):
             nouns.append(word)
nouns.sort()

In [11]:
nounx = list(dict.fromkeys(nouns))
#print(nounx)
x=len(nounx)
print(x)

1011


In [12]:
from nltk.corpus import wordnet as wn
def get_some_word_synonyms(word):
    word = word.lower()
    synonyms = []
    synsets = wordnet.synsets(word[1:])
    print(synsets)
   
for word in nounx:
    #if (synsets == 2):
    #print((word[1:]))
    synsets = get_some_word_synonyms(word)
    #print (synonyms)
    #x = len(synsets)
    #print (x)


[Synset('ten.n.01'), Synset('ten.s.01')]
[Synset('hundred.n.01'), Synset('hundred.s.01')]
[]
[]
[]
[]
[]
[]
[]
[]
[Synset('thirty.n.01'), Synset('thirty.s.01')]
[Synset('abandon.n.01'), Synset('wildness.n.01'), Synset('abandon.v.01'), Synset('abandon.v.02'), Synset('vacate.v.02'), Synset('abandon.v.04'), Synset('abandon.v.05')]
[Synset('abhorrent.s.01')]
[Synset('ability.n.01'), Synset('ability.n.02')]
[Synset('abortion.n.01'), Synset('miscarriage.n.01')]
[Synset('above.n.01'), Synset('above.s.01'), Synset('above.r.01'), Synset('above.r.02')]
[Synset('accidental.n.01'), Synset('accidental.s.01')]
[Synset('agreement.n.03'), Synset('accord.n.02'), Synset('treaty.n.01'), Synset('accord.n.04'), Synset('harmonize.v.01'), Synset('accord.v.02')]
[Synset('roll_up.v.02'), Synset('accumulate.v.02')]
[Synset('accusation.n.01'), Synset('accusation.n.02')]
[Synset('accuse.v.01'), Synset('charge.v.02')]
[Synset('accomplishment.n.01')]
[Synset('admit.v.01'), Synset('acknowledge.v.02'), Synset('notice

[Synset('bad.n.01'), Synset('bad.a.01'), Synset('bad.s.02'), Synset('bad.s.03'), Synset('bad.s.04'), Synset('regretful.a.01'), Synset('bad.s.06'), Synset('bad.s.07'), Synset('bad.s.08'), Synset('bad.s.09'), Synset('bad.s.10'), Synset('bad.s.11'), Synset('bad.s.12'), Synset('bad.s.13'), Synset('bad.s.14'), Synset('badly.r.05'), Synset('badly.r.06')]
[Synset('barroom.n.01'), Synset('bar.n.02'), Synset('bar.n.03'), Synset('measure.n.07'), Synset('bar.n.05'), Synset('prevention.n.01'), Synset('bar.n.07'), Synset('bar.n.08'), Synset('legal_profession.n.01'), Synset('stripe.n.05'), Synset('cake.n.01'), Synset('browning_automatic_rifle.n.01'), Synset('bar.n.13'), Synset('bar.n.14'), Synset('bar.n.15'), Synset('bar.v.01'), Synset('barricade.v.01'), Synset('banish.v.03'), Synset('bar.v.04')]
[Synset('base.n.01'), Synset('foundation.n.03'), Synset('base.n.03'), Synset('base.n.04'), Synset('base.n.05'), Synset('floor.n.03'), Synset('basis.n.02'), Synset('base.n.08'), Synset('nucleotide.n.01'), Sy

[Synset('check.n.01'), Synset('assay.n.01'), Synset('check.n.03'), Synset('arrest.n.02'), Synset('confirmation.n.01'), Synset('check.n.06'), Synset('check_mark.n.01'), Synset('hindrance.n.01'), Synset('check.n.09'), Synset('check.n.10'), Synset('bridle.n.02'), Synset('check.n.12'), Synset('check.n.13'), Synset('check.v.01'), Synset('check.v.02'), Synset('see.v.10'), Synset('control.v.02'), Synset('check.v.05'), Synset('check.v.06'), Synset('check.v.07'), Synset('check.v.08'), Synset('match.v.01'), Synset('check.v.10'), Synset('discipline.v.01'), Synset('check.v.12'), Synset('check.v.13'), Synset('check.v.14'), Synset('check.v.15'), Synset('check.v.16'), Synset('check.v.17'), Synset('check.v.18'), Synset('check.v.19'), Synset('check.v.20'), Synset('determine.v.08'), Synset('check.v.22'), Synset('check.v.23'), Synset('check.v.24'), Synset('crack.v.01')]
[Synset('checker.n.01'), Synset('checker.n.02'), Synset('checker.n.03'), Synset('check.v.16'), Synset('checker.v.02')]
[]
[Synset('chew.

[Synset('specify.v.03'), Synset('define.v.02'), Synset('define.v.03'), Synset('define.v.04'), Synset('specify.v.02')]
[Synset('definition.n.01'), Synset('definition.n.02')]
[Synset('democracy.n.01'), Synset('democracy.n.02'), Synset('majority_rule.n.01')]
[Synset('demographic.n.01'), Synset('demographic.a.01')]
[Synset('show.v.01'), Synset('prove.v.02'), Synset('attest.v.01'), Synset('demonstrate.v.04')]
[Synset('word_picture.n.01'), Synset('depicting.n.01'), Synset('delineation.n.02'), Synset('depiction.n.04')]
[Synset('form.n.01'), Synset('descriptor.n.02')]
[Synset('deserve.v.01')]
[Synset('desire.n.01'), Synset('desire.n.02'), Synset('desire.n.03'), Synset('desire.v.01'), Synset('hope.v.01'), Synset('desire.v.03')]
[Synset('contempt.n.01'), Synset('despite.n.02')]
[Synset('destroy.v.01'), Synset('destroy.v.02'), Synset('demolish.v.03'), Synset('destroy.v.04')]
[Synset('detach.v.01'), Synset('detach.v.02'), Synset('detach.v.03')]
[Synset('detail.n.01'), Synset('detail.n.02'), Synset

[Synset('exemption.n.01'), Synset('exemption.n.02'), Synset('exemption.n.03')]
[Synset('exist.v.01'), Synset('exist.v.02')]
[Synset('expect.v.01'), Synset('ask.v.04'), Synset('expect.v.03'), Synset('expect.v.04'), Synset('expect.v.05'), Synset('have_a_bun_in_the_oven.v.01')]
[Synset('experience.n.01'), Synset('experience.n.02'), Synset('experience.n.03'), Synset('experience.v.01'), Synset('know.v.05'), Synset('experience.v.03'), Synset('feel.v.01'), Synset('have.v.11')]
[Synset('experimental.a.01'), Synset('experimental.s.02'), Synset('experimental.s.03')]
[Synset('expert.n.01'), Synset('adept.s.01'), Synset('technical.s.04')]
[Synset('explanation.n.01'), Synset('explanation.n.02'), Synset('explanation.n.03')]
[Synset('expose.n.01'), Synset('expose.v.01'), Synset('unwrap.v.02'), Synset('expose.v.03'), Synset('uncover.v.02'), Synset('disclose.v.02'), Synset('queer.v.02'), Synset('expose.v.07'), Synset('debunk.v.01'), Synset('expose.v.09')]
[Synset('exposure.n.01'), Synset('exposure.n.02

...

[Synset('opposition.n.04'), Synset('adversary.n.01'), Synset('opponent.s.01')]
[Synset('oppose.v.01'), Synset('fight.v.02'), Synset('oppose.v.03'), Synset('pit.v.01'), Synset('react.v.02'), Synset('oppose.v.06')]
[Synset('organ.n.01'), Synset('organ.n.02'), Synset('electric_organ.n.01'), Synset('organ.n.04'), Synset('organ.n.05'), Synset('harmonium.n.01')]
[Synset('organization.n.01'), Synset('arrangement.n.03'), Synset('administration.n.02'), Synset('organization.n.04'), Synset('organization.n.05'), Synset('organization.n.06'), Synset('constitution.n.02')]
[]
[Synset('ottoman.n.01'), Synset('ottoman.n.02'), Synset('ottoman.n.03'), Synset('footstool.n.01'), Synset('ottoman.a.01')]
[]
[Synset('result.n.03'), Synset('consequence.n.01')]
[Synset('outside.n.01'), Synset('outside.n.02'), Synset('outside.a.01'), Synset('external.s.02'), Synset('outside.s.03'), Synset('outdoor.a.01'), Synset('outside.s.05'), Synset('outside.s.06'), Synset('external.s.03'), Synset('outside.s.08'), Synset('outs

[Synset('pull.n.01'), Synset('pull.n.02'), Synset('pull.n.03'), Synset('pull.n.04'), Synset('wrench.n.01'), Synset('puff.n.07'), Synset('pull.n.07'), Synset('pull.v.01'), Synset('attract.v.01'), Synset('pull.v.03'), Synset('pull.v.04'), Synset('perpetrate.v.01'), Synset('draw.v.05'), Synset('pull.v.07'), Synset('pull.v.08'), Synset('pull.v.09'), Synset('pull.v.10'), Synset('pull.v.11'), Synset('rend.v.01'), Synset('pull.v.13'), Synset('pluck.v.05'), Synset('extract.v.01'), Synset('pull.v.16'), Synset('pull.v.17')]
[Synset('pump.n.01'), Synset('heart.n.02'), Synset('pump.n.03'), Synset('pump.v.01'), Synset('pump.v.02'), Synset('pump.v.03'), Synset('pump.v.04'), Synset('pump.v.05'), Synset('pump.v.06'), Synset('pump.v.07'), Synset('pump.v.08')]
[Synset('put_option.n.02'), Synset('put.v.01'), Synset('put.v.02'), Synset('frame.v.04'), Synset('put.v.04'), Synset('invest.v.01'), Synset('place.v.11'), Synset('put.v.07'), Synset('put.v.08'), Synset('arrange.v.07')]
[Synset('putin.n.01')]
[Syns

[Synset('routine.n.01'), Synset('act.n.04'), Synset('routine.n.03'), Synset('everyday.s.01')]
[Synset('severe_acute_respiratory_syndrome.n.01')]
[Synset('satisfy.v.01'), Synset('satisfy.v.02'), Synset('meet.v.04')]
[Synset('say.n.01'), Synset('state.v.01'), Synset('allege.v.01'), Synset('suppose.v.01'), Synset('read.v.02'), Synset('order.v.01'), Synset('pronounce.v.01'), Synset('say.v.07'), Synset('say.v.08'), Synset('say.v.09'), Synset('say.v.10'), Synset('say.v.11')]
[Synset('scale.n.01'), Synset('scale.n.02'), Synset('scale.n.03'), Synset('scale.n.04'), Synset('scale.n.05'), Synset('scale.n.06'), Synset('scale.n.07'), Synset('scale.n.08'), Synset('plate.n.14'), Synset('scale.n.10'), Synset('scale.v.01'), Synset('scale.v.02'), Synset('scale.v.03'), Synset('scale.v.04'), Synset('scale.v.05'), Synset('scale.v.06'), Synset('scale.v.07'), Synset('scale.v.08')]
[Synset('chilling.s.01')]
[Synset('scene.n.01'), Synset('scene.n.02'), Synset('view.n.02'), Synset('scene.n.04'), Synset('picture

[Synset('supernatural.n.01'), Synset('supernatural.a.01')]
[Synset('supply.n.01'), Synset('supply.n.02'), Synset('provision.n.02'), Synset('supply.v.01'), Synset('issue.v.02'), Synset('provide.v.02'), Synset('add.v.02')]
[Synset('support.n.01'), Synset('support.n.02'), Synset('support.n.03'), Synset('support.n.04'), Synset('documentation.n.03'), Synset('support.n.06'), Synset('support.n.07'), Synset('support.n.08'), Synset('accompaniment.n.02'), Synset('support.n.10'), Synset('support.n.11'), Synset('support.v.01'), Synset('support.v.02'), Synset('back.v.01'), Synset('hold.v.10'), Synset('confirm.v.01'), Synset('subscribe.v.03'), Synset('corroborate.v.03'), Synset('defend.v.01'), Synset('support.v.09'), Synset('patronize.v.04'), Synset('digest.v.03')]
[Synset('supporter.n.01'), Synset('patron.n.03'), Synset('assistant.n.01'), Synset('garter.n.01'), Synset('athletic_supporter.n.01')]
[Synset('certain.a.02'), Synset('certain.s.07'), Synset('certain.a.04'), Synset('sure.s.04'), Synset('ce

[Synset('tweet.n.01'), Synset('tweet.v.01'), Synset('pinch.v.01')]
[Synset('twin.n.01'), Synset('gemini.n.01'), Synset('twin.n.03'), Synset('counterpart.n.02'), Synset('twin.v.01'), Synset('match.v.03'), Synset('twin.v.03'), Synset('twin.v.04'), Synset('duplicate.s.02')]
[Synset('two.n.01'), Synset('deuce.n.04'), Synset('two.s.01')]
[Synset('unidentified_flying_object.n.01')]
[Synset('united_kingdom.n.01')]
[Synset('ukraine.n.01')]
[Synset('ultimate.n.01'), Synset('ultimate.a.01'), Synset('ultimate.s.02')]
[]
[Synset('underdog.n.01')]
[Synset('sabotage.v.01'), Synset('cave.v.01')]
[Synset('understand.v.01'), Synset('understand.v.02'), Synset('understand.v.03'), Synset('understand.v.04'), Synset('sympathize.v.02')]
[]
[Synset('unintentional.s.01'), Synset('unintentional.s.02')]
[Synset('universal.n.01'), Synset('universal.n.02'), Synset('universal.n.03'), Synset('universal_joint.n.01'), Synset('cosmopolitan.s.03'), Synset('universal.s.02'), Synset('universal.s.03')]
[Synset('university.

In [13]:
Average_data= (889/1011*100)
print (Average_data)

87.93273986152325


In [14]:
from nltk.corpus import wordnet as wn
rws['synsets'] = rws['lemmatized'].apply(lambda x: [wn.synsets(word) for (word) in x])
rws.head()

,Tweets,Tweets_token,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized,synsets
0,"#USER# #USER# #USER# #USER# #USER# I'd be shocked if he ""pushed"" a theory that there was little ...","[#USER#, #USER#, #USER#, #USER#, #USER#, I would, be, shocked, if, he, ""pushed"", a, theory, that...","#USER# #USER# #USER# #USER# #USER# I would be shocked if he ""pushed"" a theory that there was lit...","[#, USER, #, #, USER, #, #, USER, #, #, USER, #, #, USER, #, I, would, be, shocked, if, he, ``, ...","[#, user, #, #, user, #, #, user, #, #, user, #, #, user, #, i, would, be, shocked, if, he, ``, ...","[user, user, user, user, user, i, would, be, shocked, if, he, ``, pushed, '', a, theory, that, t...","[i, would, be, shocked, if, he, ``, pushed, '', theory, that, there, was, little, to, no, eviden...","[(i, NN), (would, MD), (be, VB), (shocked, VBN), (if, IN), (he, PRP), (``, ``), (pushed, JJ), ('...","[(i, n), (would, n), (be, v), (shocked, v), (if, n), (he, n), (``, n), (pushed, a), ('', n), (th...","[i, would, be, shocked, if, he, ``, pushed, '', theory, that, there, wa, little, to, no, evidenc...","[[Synset('iodine.n.01'), Synset('one.n.01'), Synset('i.n.03'), Synset('one.s.01')], [], [Synset(..."
1,#USER# #USER# Maybe that was off set by his incompetency and danger to global stability? idk,"[#USER#, #USER#, Maybe, that, was, off, set, by, his, incompetency, and, danger, to, global, sta...",#USER# #USER# Maybe that was off set by his incompetency and danger to global stability? i do no...,"[#, USER, #, #, USER, #, Maybe, that, was, off, set, by, his, incompetency, and, danger, to, glo...","[#, user, #, #, user, #, maybe, that, was, off, set, by, his, incompetency, and, danger, to, glo...","[user, user, maybe, that, was, off, set, by, his, incompetency, and, danger, to, global, stabili...","[maybe, that, was, off, set, by, his, incompetency, and, danger, to, global, stability, i, do, n...","[(maybe, RB), (that, DT), (was, VBD), (off, RB), (set, VBN), (by, IN), (his, PRP$), (incompetenc...","[(maybe, r), (that, n), (was, v), (off, r), (set, v), (by, n), (his, n), (incompetency, n), (and...","[maybe, that, wa, off, set, by, his, incompetency, and, danger, to, global, stability, i, do, no...","[[Synset('possibly.r.01')], [], [Synset('washington.n.02')], [Synset('murder.v.01'), Synset('off..."
2,"#USER# #USER# Select doctors d not represent the medical community. Also, does that mean that re...","[#USER#, #USER#, Select, doctors, d, not, represent, the, medical, community., Also,, does, that...","#USER# #USER# Select doctors d not represent the medical community. Also, does that mean that re...","[#, USER, #, #, USER, #, Select, doctors, d, not, represent, the, medical, community, ., Also, ,...","[#, user, #, #, user, #, select, doctors, d, not, represent, the, medical, community, ., also, ,...","[user, user, select, doctors, d, not, represent, the, medical, community, also, does, that, mean...","[select, doctors, d, not, represent, medical, community, also, does, that, mean, that, restricti...","[(select, JJ), (doctors, NNS), (d, VBP), (not, RB), (represent, JJ), (medical, JJ), (community, ...","[(select, a), (doctors, n), (d, v), (not, r), (represent, a), (medical, a), (community, n), (als...","[select, doctor, d, not, represent, medical, community, also, doe, that, mean, that, restriction...","[[Synset('choose.v.01'), Synset('choice.s.01'), Synset('blue-ribbon.s.01')], [Synset('doctor.n.0..."
3,#USER# #USER# #USER# #USER# That there are complicated cases does not invalidate the point. To s...,"[#USER#, #USER#, #USER#, #USER#, That, there, are, complicated, cases, does, not, invalidate, th...",#USER# #USER# #USER# #USER# That there are complicated cases does not invalidate the point. To s...,"[#, USER, #, #, USER, #, #, USER, #, #, USER, #, That, there, are, complicated, cases, does, not...","[#, user, #, #, user, #, #, user, #, #, user, #, that, there, are, complicated, cases, does, not..."

In [15]:
rws.to_csv('C:/Users/Ankan SInha/Desktop/Finl 2/Data/Trainng Set/i/User_1.csv')

In [16]:
Average_data= (889/1011*100)
print (Average_data)

87.93273986152325


In [17]:
Freq_of_homonyms_in_the_tweet = (889/200)
print (Freq_of_homonyms_in_the_tweet,'%')

4.445 %


In [2]:
from sklearn import metrics
# Predicted values
y_pred = [24, 32]
# Actual values
y_act = [76, 68]
# Printing the confusion matrix
# The columns will show the instances predicted for each label,
# and the rows will show the actual number of instances for each label.
print(metrics.confusion_matrix(y_act, y_pred, labels=["a", "b"]))
# Printing the precision and recall, among other metrics
print(metrics.classification_report(y_act, y_pred, labels=["a", 
"b","c"]))

ValueError: At least one label specified must be in y_true